<a href="https://colab.research.google.com/github/Thelmaandherls/AI-ML/blob/Udemy/Fine_Tuning_LLMs_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning LLMs with Hugging Face

## Step 1: Installing and importing the libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curren

In [ ]:
!pip install huggingface_hub

In [ ]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [ ]:
#AutoModelForCausalLM class - each token is predicted based on preciding model - using dfrom_pretrained method
#decreasing memeory by enabling 4 bit precision thf model weighting are loaded in 4 bit format as opposed to 8 bit -> dec model size thf dec usage - use quantization
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="aboonaji/llama2finetune-v2",
                                                   quantization_config= BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype= getattr(torch, "float16"), bnb_4bit_quant_type= "nf4") )
#getatrr - can fecth float16 data type from torch module as when loading finetunellm2 model, will load 2 pytorch moduke
#nf4 - want nf4 quantitzation data type in weights/linear layers of llm
#dec memory usage and speed up computations
llama_model.config.use_cache = False #not storing output of rpeviously computes layers
#deactivate the more accurtely compuation layer of linear layers thf speeding it up
llama_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Step 3: Loading the tokenizer

In [ ]:
#loading the tokenizer is compatible with llm model whislt ensuring the tokeniser uses same special tokens and passing (important)
llama_tokeniser = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="aboonaji/llama2finetune-v2", trust_remote_code = True)
#ensure back toke is the same as the end of sequence token
#config padding to make sequences of the same length to process batches of text data
llama_tokeniser.pad_token = llama_tokeniser.eos_token
#rught padding
llama_tokeniser.padding_side = "right"

## Step 4: Setting the training arguments

In [ ]:
#need to set sperately as in step 5, it takes xx args, one of which is this
#config training params of future trianing that will happen once llm2 model has been retrained w new trianing model w new dataset containing medical terms
#dec trianigbatch size to 4 - thf model will process 4 trianing exmaples in ehc trainign iteration which won't overlaod my memory
training_args = TrainingArguments(output_dir = "./results", per_device_train_batch_size= 4, max_steps = 100)

## Step 5: Creating the Supervised Fine-Tuning trainer

In [16]:
#1of the 2 techniques to dev llm for langauge augumentation - other is reinforcement learning from human feedback
#trasnfer learning technique - weights of pre-trianed model are trained on new data
trainer = SFTTrainer(model= llama_model,
                     args = training_args,
                     train_dataset = load_dataset(path="aboonaji/wiki_medical_terms_llam2_format", split = "train"),
                     tokenizer = llama_tokeniser,
                     peft_config= LoraConfig(task_type= "CAUSAL_LM", r=64, lora_alpha= 16, lora_dropout= 0.1),
                     dataset_text_field = "text")
#aparam effcient fne tuning - dec #params

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

## Step 6: Training the model

In [ ]:
#augumenting knoledge of llm
"""
What is going on:
max num of 100 steps
fine tunign process is occuring - weights of pre-trianiend llm2 model are beign retrianed on new data abut medical terms
4 bit qunatization and param efficient fine tuning with lora technique so only effciitn ones w min tuning using low_rankadaptive matrices usign lora tchnique
tokeniser is doing the padding to ensure all sequences are the same length using the end of sequence token
lora and quantiziation prevents my gpu reachign it's capacity
"""
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, *

Step,Training Loss


## Step 7: Chatting with the model

In [ ]:
user_prompt = ""
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokeniser, max_length = 300)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]["generated_text"])